In [459]:
import sympy as sp
import numpy as np
from IPython.display import display, Math, Latex
import codecs

# Introduction

In this notebook a step-by-step symbolic solution to obtain the kernel $K(x|u)$ for the problem

$f(x)(1+W(x) + W(x+1)) - W(x)f(x-1) - W(x+1)f(x+1) = g(x)$

when  the weights are given by a barrier at $x= 0$ with 

$W(x) = \left\{\begin{matrix}
\lambda\,, & x\neq 0\\
\mu, & x=0\\
\end{matrix}\right.$

We first show the general form of the solution, which is extremely extense, and then we show how it can be greatly simplified in the case 

$\lambda=2\,, \quad \mu = 2 \, w_0$

The kernel is shown to be $K(x|u) = a(x)a(u)e^{-|t(x)-t(u)|}$, where $t(x)$ is given implicitly by $\Delta t(x) = t(x)-t(x-1)$. We compute explicit solutions for $a(x)$ and $\Delta t(x)$.

$ $
t.$

# General solution for the Barrier at $x_0 = 0$

We begin by defining the weight function W(x)

In [462]:
x= sp.symbols('x')
w0= sp.symbols('w0')
λ= sp.symbols('λ')
n = sp.symbols('n')
μ = sp.symbols('μ')

W = sp.Piecewise(
    (λ, x <= -1),
    (μ, (x>-1)&(x<1)),
    (λ, x >= 1),
)

display(Latex(r"$W(x) = $ " + sp.latex(W, mode='inline')))

<IPython.core.display.Latex object>

# Explicit barrier domain transform by symbolic computing

The $γ(λ)$ is the default value for the $B(x)$ coefficients in homogeneous regions, computed by continued fractions

$B(x) = 1 + W(x) + W(x+1)  - W(x+1)^2/B(x+1)$

$B(x) = 1 + λ + λ  - λ^2/B(x+1)$

$B(x) = 1 + 2λ - \frac{λ^2}{ 1 + 2λ - \frac{λ^2}{ 1 + 2λ \ldots}} = γ(λ)$

# Computing B

If, however, $B(x)$ is not $\textbf{all}$ equal to $\gamma(\lambda)$, then we can still compute $B(x-k)$ for arbitrary choice of $k$ using matrix multiplication. The idea is as follows:

Suppose $B(x) = \frac{p}{q}$ for some fraction $p/q$, and that the weights are all homogeneous for $x$ in $x+1$ to $-\infty$.

$B(x-1) = 1 + 2\lambda - \frac{\lambda^2}{\frac{p}{q}}$

is simplified to

$B(x-1) = \frac{νp- λ^2q}{p}$, which means each step $x \mapsto x-1$ is written in matrix form as 

$      \begin{bmatrix}
\nu  & -\lambda^2\\ 
1 & 0
\end{bmatrix}
\begin{bmatrix}
p\\ 
r
\end{bmatrix} \mapsto \begin{bmatrix}
\nu p - \lambda^2 r\\ 
p
\end{bmatrix}\,,$

Therefore $B(x-k)$ for $k>=0$ can be written in terms of $B(x) = \frac{p}{q}$ with

$  \frac{\text{numerator}(B(x-k))}{\text{denominator}(B(x-k))}  =   \begin{bmatrix}
\nu  & -\lambda^2\\ 
1 & 0
\end{bmatrix}^k
\begin{bmatrix}
p\\ 
r
\end{bmatrix} $


We begin my writing down the variables

In [397]:
γ =  (2 * λ + sp.sqrt(4 * λ + 1) + 1)/2
ν = 1 + 2*λ
Bstep_mat = sp.Matrix([[ν, -λ*λ], [1, 0]])

display(Latex(r"$γ = $ " + sp.latex(γ, mode='inline')))
display(Latex(r"$ν = $ " + sp.latex(ν, mode='inline')))
display(Bstep_mat)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Matrix([
[2*λ + 1, -λ**2],
[      1,     0]])

## Diagonalizing B(x-k)

In [407]:
k = sp.symbols('k')
bstart = sp.symbols('bs')
P, D = Bstep_mat.diagonalize()
Pinv = P.inv()

Bsteps_mat = (P * (D**k) * P.inv())*sp.Matrix([[bstart], [1]]) #bstart is the starting value of b(x) to compute b(x-k)

Bsteps = Bsteps_mat[0]/Bsteps_mat[1]

#the starting fracion can be chosen conveniently, we simply make q=1 to simplify computation
display(Latex(r"We then have the expression, starting from $B(-1)$ for decreasing indices"))

display(Latex(r" $B(-k) =$ " + sp.latex(Bsteps.subs(bstart, sp.symbols('B(-1)')), mode='inline')))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## B as a pure function

We write down the function $B(x)$ as a piecewise function. Notice that it is constant equal to $\gamma(\lambda)$ for $x>0$. Then

\begin{matrix}
B(1) & =  \gamma(\lambda)\\ 
B(0) & =  1 + \mu + \lambda - \frac{\lambda^2}{B(1)}\\
B(-1) & = 1 + \lambda + \mu - \mu^2/B(0)\\
B(-2) & = 1 + 2\lambda - \lambda^2/B(0)
\end{matrix}
}
$

In [411]:
Bx1 = γ
Bx0 = 1 + W.subs(x,0)+ W.subs(x,1) - W.subs(x,1)**2/Bx1
Bxm1 =  1 + W.subs(x,-1)+ W.subs(x,0) - W.subs(x,0)**2/Bx0


B = sp.Piecewise(
    (Bx1, x >= 1),
    (Bx0 , (x < 1)&(x>-1)), #at x==0
    (Bsteps.replace(k,-x-1).replace(bstart, Bxm1), x<=-1)
)

display(Latex(r"We obtain the large expression"))

display(Latex(r" $B(x) =$ " + sp.latex(B, mode='inline')))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Diagonalizing B2

Similarly we can solve for $B_2(x) = 1 + W(x) + W(x+1) - W(x)^2/B_2(x-1)$. The key difference is that we are computing a forward step $B_2(x+k)$ as a function of $B(x)$. The steps are taken at the right side of the barrier. Since the weights are all $\lambda$ except at $x_0=0$, then the steps are computed with the same matrix used to compute $B$.

In [420]:
bstart2 = sp.symbols('bs2')
B2steps = Bsteps.replace(bstart,bstart2)

display(Latex(r"We then have the expression, starting from $B_2(1)$ for increasing indices"))
display(Latex(r" $B_2(k) =$ " + sp.latex(B2steps.subs(bstart2,sp.symbols(r'B_{2}(1)')), mode='inline')))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## B2 as a pure function

We write down the function $B_2(x)$ as a piecewise function. Notice that it is constant equal to $\gamma(\lambda)$ for $x<=2$. Then

\begin{matrix}
B_2(-2) & =  \gamma(\lambda)\\ 
B_2(-1) & =  1 + \lambda + \mu - \frac{\lambda^2}{B_2(-2)}\\
B_2(0) & = 1 + \mu + \lambda - \frac{\mu^2}{B_2(-1)}
\\
B_2(1) & = 1 + 2\lambda - \frac{\lambda^2}{B_2(0)}
\end{matrix}


In [421]:
B2xm2 = γ
B2xm1 = 1 + W.subs(x,-1)+ W.subs(x,0) - W.subs(x,-1)**2/B2xm2
B2x0 =  1 + W.subs(x,0)+ W.subs(x,1) - W.subs(x,0)**2/B2xm1


B2 = sp.Piecewise(
    (B2xm2, x <= -2),
    (B2xm1, (x < 0)&(x>-2)), #at x==-1
    (B2steps.replace(k,x).replace(bstart2, B2x0), x>=0)
)

display(Latex(r"So"))

display(Latex(r" $B_2(x) =$ " + sp.latex(B2, mode='inline')))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

$B_2$ and $B$ satisfy the symmetry

$B_2(-1+j) = B(-j)$

This can be seen at numerically

In [424]:
for j in range(-5,5):
    print((B2.subs(x, -1+j) - B.subs(x, -j)).subs({
            λ : 10,
            μ : 3,
        }
    ).evalf())

0
0
0
0
0
0
0
0
0
0


# Squared normalization coefficients $a^2$ and Domain transform dt

We use that 

$ a(x)^2 = \frac{1}{B(x) - \frac{W(x)^2}{B_2(x-1)}}\,.$

and

$ (t(x)-t(x-1)) = -\log \left( \frac{-1 + \sqrt{4 a(x-1)^2 a(x)^2W(x)^2+1}}{2 a(x-1)a(x)W(x)}\right)$

In [425]:
asqr = 1/(B- W*W/(B2.replace(x,x-1)))

# Domain transform

The domain transform is then given by

In [426]:
dt = sp.log((2*sp.sqrt(asqr.replace(x,x-1)*asqr)*W)/(-1 + sp.sqrt(4*asqr.replace(x,x-1)*asqr*W*W+1)))
display(dt)

log(2*sqrt(1/((-Piecewise((λ**2, x <= -1), (μ**2, x < 1), (λ**2, True))*Piecewise((1/(λ + sqrt(4*λ + 1)/2 + 1/2), x <= -1), (1/(-λ**2/(λ + sqrt(4*λ + 1)/2 + 1/2) + λ + μ + 1), x < 1), (((-(λ - sqrt(4*λ + 1)/2 + 1/2)**(x - 1)/sqrt(4*λ + 1) + (λ + sqrt(4*λ + 1)/2 + 1/2)**(x - 1)/sqrt(4*λ + 1))*(λ - μ**2/(-λ**2/(λ + sqrt(4*λ + 1)/2 + 1/2) + λ + μ + 1) + μ + 1) + (-2*λ + sqrt(4*λ + 1) - 1)*(λ + sqrt(4*λ + 1)/2 + 1/2)**(x - 1)/(2*sqrt(4*λ + 1)) + (λ - sqrt(4*λ + 1)/2 + 1/2)**(x - 1)*(2*λ + sqrt(4*λ + 1) + 1)/(2*sqrt(4*λ + 1)))/((-(λ - sqrt(4*λ + 1)/2 + 1/2)*(λ - sqrt(4*λ + 1)/2 + 1/2)**(x - 1)/sqrt(4*λ + 1) + (λ + sqrt(4*λ + 1)/2 + 1/2)*(λ + sqrt(4*λ + 1)/2 + 1/2)**(x - 1)/sqrt(4*λ + 1))*(λ - μ**2/(-λ**2/(λ + sqrt(4*λ + 1)/2 + 1/2) + λ + μ + 1) + μ + 1) + (-2*λ + sqrt(4*λ + 1) - 1)*(λ + sqrt(4*λ + 1)/2 + 1/2)*(λ + sqrt(4*λ + 1)/2 + 1/2)**(x - 1)/(2*sqrt(4*λ + 1)) + (λ - sqrt(4*λ + 1)/2 + 1/2)*(λ - sqrt(4*λ + 1)/2 + 1/2)**(x - 1)*(2*λ + sqrt(4*λ + 1) + 1)/(2*sqrt(4*λ + 1))), True)) + Piecewi

# Domain Transform at $x_0$

In the expression

$ (t(0)-t(-1)) = -\log \left( \frac{-1 + \sqrt{4 a(-1)^2 a(0)^2W(0)^2+1}}{2 a(-1)a(0)W(0)}\right)$

Using the substitution  $\lambda = n(n+1)$ and $\mu = w_0 n (n+1)$ we can greatly simplify the expressions. Define 

$U := 4 a(-1)^2a(0)^2W(0)^2$

$ (t(0)-t(-1)) = -\log \left( \frac{-1 + \sqrt{U+1}}{\sqrt{U}}\right)$

In [429]:
n = sp.Symbol('n',positive=True)
w0 = sp.Symbol('w0',positive=True)

U = ((4*asqr.subs(x,-1)*asqr.subs(x,0)*W.subs(x,0)**2)) #evaluate at 0
U = U.subs({
    λ:n*(n+1),
    μ:w0*n*(n+1),
})
U = U.subs({
    4*n*(n+1)+1: (2*n+1)**2, #simplifies square root terms
}   
).simplify()

U = U.subs({
    n**2 * w0**2 + 2*n *w0 +1 : (n*w0 + 1)**2, #simplifies square root terms
    4*n**2 * w0**2 + 4*n *w0 +1 : (2*n*w0 + 1)**2, #simplifies square root terms
})

display(Latex(r"$U = $ " + sp.latex(U, mode='inline')))


<IPython.core.display.Latex object>

We then get the expression at the origin

In [437]:
dt_at_0 = (-sp.log((-1 + sp.sqrt(U + 1))/sp.sqrt(U))).simplify()

display(Latex(r"$\Delta t(0) = $ " + sp.latex(dt_at_0, mode='inline')))

<IPython.core.display.Latex object>

We further simplify this with

$4 n^2  w_0^2 (n w_0+1)^2 + (2 n w_0 + 1)^2 \mapsto (1 + 2 n w_0(1 + n w_0))^2$

In [441]:
dt_at_0 = dt_at_0.subs({
    ((4*n**2 * w0**2 *(n*w0+1)**2 + (2*n*w0 + 1)**2)) : (1 + 2*n*w0*(1 + n*w0))**2,
}).expand().simplify()

display(Latex(r"$\Delta t(0) = $ " + sp.latex(dt_at_0, mode='inline')))

<IPython.core.display.Latex object>

# Solutions for the barrier with $\lambda=2$

We begin by obtaining a simplified formula for $a(x)^2$

In [373]:
W_2 = W.subs({
  λ : 2,
  μ: w0*2,
})

asqr_2 = asqr.subs({
  λ : 2,
  μ: w0*2,
}).simplify()

display(Latex(r"$a(x)^2 = $ " + sp.latex(asqr_2, mode='inline')))


<IPython.core.display.Latex object>

We make better simplifications by defining dt as a new piecewise function to satisfy

$ (t(x)-t(x-1)) = -\log \left( \frac{-1 + \sqrt{4 a(x-1)^2 a(x)^2W(x)^2+1}}{2 a(x-1)a(x)W(x)}\right)$

Extract the piecewise parts of $a(x)$

In [374]:
xpos = sp.Symbol('x',positive=True)
#extract the part x>=0
asqr_2_above_0 = asqr_2.subs({
 x : x+1
}
).subs(x, xpos).subs(xpos,x-1)
asqr_2_above_0

(-4*2**(2*x)*w0 - 2*2**(2*x) + w0 - 1)/(2*(2*2**(2*x)*w0 - 8*4**x*w0 - 3*4**x))

In [375]:
#extract the part x<0
asqr_2_below_0 = asqr_2.subs({
 x : -x-1
}
).subs(x, xpos).subs(xpos, -x-1)
asqr_2_below_0

(-2**(2*x + 1)*w0 + 2**(2*x + 1) + 2*w0 + 1)/(3*(2*w0 + 1))

We can observe the identity $a(x) = a(-x-1)$

In [376]:
(asqr_2_above_0.subs(x,x) - asqr_2_below_0.subs(x,-x-1)).simplify()

0

We can also simplify $a(x)$ to 

In [519]:
asqr_2_above_0_simplified = asqr_2_above_0.subs(2**(2*x),4**x) #replace 2^2x by 4^x
asqr_2_above_0_simplified = ((-sp.numer(asqr_2_above_0_simplified)/(4**x)).expand()/
                             (-(sp.denom(asqr_2_above_0_simplified)/(4**x)).expand())
                            )
asqr_2_simplified = sp.Piecewise(
    (asqr_2_above_0_simplified, x >=0),
    (asqr_2_above_0_simplified.subs(x,-x-1), x<0)
)


display(Latex(r"$a(x)^2 = $ " + sp.latex(asqr_2_simplified, mode='inline')))

<IPython.core.display.Latex object>

This means that the domain transform is symmetric, because it is composed of terms $a(x)a(x-1)W(x)$, to which we apply the identity $a(x) = a(-x-1)$, and use that $W(x)$ is symmetric

$a(x)a(x-1)W(x) = a(-x-1)a(-(x-1)-1)W(-x) = a(-x-1)a(-x)W(x)$

This confirms the symmetry of the domain transform. We then only need to find expressions for positive $x$ for the transform.

In [381]:
#define dt(x) for x>0
dt_2_above_0 = -sp.log((-1 + sp.sqrt(4*asqr_2_above_0.subs(x,x-1)*asqr_2_above_0 * W_2.subs(x,1)**2+1))/(
        2*sp.sqrt(asqr_2_above_0.subs(x,x-1)*asqr_2_above_0) * W_2.subs(x,-1)))

display(Latex(r"$\Delta t(x) = $ " + sp.latex(dt_2_above_0, mode='inline')))

display(Latex("We will show how to simplify this expression to"))

Math('\\frac{\\log{\\left(\\frac{6 \\left(w_{0} - 1\\right)}{2 \\cdot 4^{k} w_{0} + 4^{k} - 2 w_{0} + 2} + 4 \\right)}}{2}')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

In [442]:
#define dt(x) for x>0
dt_2_above_0 = -sp.log((-1 + sp.sqrt(4*asqr_2_above_0.subs(x,x-1)*asqr_2_above_0 * W_2.subs(x,1)**2+1))/(
        2*sp.sqrt(asqr_2_above_0.subs(x,x-1)*asqr_2_above_0) * W_2.subs(x,-1)))
display(Latex("We begin with"))

display(Latex(r"$\Delta t(x) = $ " + sp.latex(dt_2_above_0, mode='inline')))



<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [443]:
display(Latex("By replacing $4^k$ with a variable $z$, we get"))
z = sp.Symbol('z', positive=True)
#exp_dt_2_above_0 = sp.exp(-dt_2_above_0)
exp_dt_2_above_0 = sp.exp(dt_2_above_0)
exp_dt_2_above_0 = exp_dt_2_above_0.subs({
    2**(2*x): z,
    4**x : z
}).simplify()

display(Latex(r"$\Delta t(x) = $ " + sp.latex(sp.log(exp_dt_2_above_0), mode='inline')))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [446]:
display(Latex("The polynomial in $z$ inside the square-root term in the denominator is"))
Pz = (((sp.denom(exp_dt_2_above_0)) +3*z + 6*w0*z)**2)
display(Latex(r"$P(z) = $ " + sp.latex(Pz, mode='inline')))

roots = list(sp.roots(Pz.expand(),z).keys()) #extract the roots in w0 as a list
root = roots[0]

display(Latex("This polynomial has two equal roots given by " +  sp.latex(root, mode='inline')))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [447]:
multiplier = sp.collect(Pz.expand(),z,evaluate=False)[z**2] #collect the term multiplying z^2 in the polynomial

display(Latex("This means we can write it as $multiplier(z- root)^2$ in the form"))

display(Latex("$P(z) = $ " +  sp.latex(multiplier*(z-root)**2, mode='inline')))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [449]:
display(Latex("Replacing $4w_0^2 + 4w_0 +1$ by $(2w_0+1)^2$ , the polynomial becomes"))
display(Latex("$P(z) = $ " +  sp.latex((multiplier*(z-root)**2).simplify().subs(4*w0**2 + 4*w0 + 1, (2*w0+1)**2), mode='inline')))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [453]:
exp_dt_2_above_0_simplified = exp_dt_2_above_0.subs({
sp.sqrt(Pz) :  sp.sqrt(multiplier)*(z-root) #compute sqrt by hand
}).simplify().subs({
     sp.sqrt(4*w0**2 + 4*w0 +1) : (2*w0+1)

})

display(Latex("Then the domain transform simplifies to"))

display(Latex(r"$\Delta t(x) = $ " + sp.latex(sp.log(exp_dt_2_above_0_simplified), mode='inline')))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [454]:
#we further simplify by replacing 4w0^2 + 4w0 +1 by (2w0+1)^2
exp_dt_2_above_0_simplified = exp_dt_2_above_0_simplified.subs({
     sp.sqrt(4*w0**2 + 4*w0 +1) : (2*w0+1)

}).simplify()

display(Latex("This is rewritten as"))

display(Latex(r"$\Delta t(x) = $ " + sp.latex(sp.log(exp_dt_2_above_0_simplified), mode='inline')))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [455]:
display(Latex("Placing all terms within the square-root"))

X=(exp_dt_2_above_0_simplified**2).expand().together()
X = sp.sqrt(sp.numer(X).expand()/sp.denom(X))

exp_dt_2_above_0_simplified = X

display(Latex(r"$\Delta t(x) = $ " + sp.latex(sp.log(exp_dt_2_above_0_simplified), mode='inline')))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [456]:
display(Latex(r"The inner part of the square-root has the form $\frac{num}{den}$, which we replace with "))
display(Latex(r"$\frac{num}{den} \mapsto \frac{num-4den}{den} + 4$"))
display(Latex(r"To obtain"))

X = exp_dt_2_above_0_simplified**2

exp_dt_2_above_0_simplified = sp.sqrt(((sp.numer(X)-4*sp.denom(X))/sp.denom(X)).simplify() + 4)

display(Latex(r"$\Delta t(x) = $ " + sp.latex(sp.log(exp_dt_2_above_0_simplified), mode='inline')))



<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [458]:
display(Latex(r"Then"))

dt_2_above_0_simplified = (sp.Rational(1,2)* sp.log(exp_dt_2_above_0_simplified**2)).subs(z, 4**k)

display(Latex(r"$\Delta t(x) = $ " + sp.latex(dt_2_above_0_simplified, mode='inline')))

display(Latex(r"As desired"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>